In [146]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib
import seaborn as sns
import csv
import re 
from matplotlib import pyplot as plt
from sklearn import ensemble, cross_validation, learning_curve, metrics , tree, naive_bayes, utils
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.datasets import load_digits, load_breast_cancer, load_iris
from sklearn.cross_validation import cross_val_score, KFold, LeaveOneOut, ShuffleSplit, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier, BaggingRegressor
from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans, AgglomerativeClustering, FeatureAgglomeration
from sklearn.preprocessing import StandardScaler, scale, OneHotEncoder
from sklearn.grid_search import GridSearchCV
matplotlib.style.use('ggplot')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


//anaconda/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['step', 'clf']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [5]:
location = '../kaggle_task/'
events = pd.DataFrame.from_csv(location+"user_activity.csv",index_col=None)
structure = pd.DataFrame.from_csv(location+"structure.csv",index_col=None)
targets = pd.DataFrame.from_csv(location+"targets.csv",index_col=None)
events_test = pd.DataFrame.from_csv(location+"user_activity_test.csv",index_col=None)

In [20]:
col_step_cost = events.groupby(by='user_id').sum().step_cost
col_step_cost_test = events_test.groupby(by='user_id').sum().step_cost

In [55]:
def dict_passed(type_of_check):
    tmp = events[events['step_type'] == type_of_check]
    passed = tmp[tmp['action'] == 'passed']
    n_passed = {}
    for user_id, _, step_id, __, ____, step_cost in passed.values:
        if user_id not in n_passed:
            n_passed[user_id] = step_cost
        else:
            n_passed[user_id] += step_cost

    tmp_test = events_test[events_test['step_type'] == type_of_check]
    passed_test = tmp_test[tmp_test['action'] == 'passed']
    n_passed_test = {}
    for user_id, _, step_id, __, ____, step_cost in passed_test.values:
        if user_id not in n_passed_test:
            n_passed_test[user_id] = step_cost
        else:
            n_passed_test[user_id] += step_cost
    return [n_passed, n_passed_test]

In [56]:
n_passed_code, n_passed_code_test = dict_passed('code')
n_passed_choice, n_passed_choice_test = dict_passed('choice')
n_passed_number, n_passed_number_test = dict_passed('number')
n_passed_string, n_passed_string_test = dict_passed('string')

In [59]:
events[events['action'] == 'started_attempt'].step_type.unique()

array(['choice', 'number', 'code', 'string'], dtype=object)

In [50]:
tmp = events[events['step_type'] == 'string']
print(len(list(events.user_id.unique())))
string_passed = tmp[tmp['action'] == 'passed']
n_passed_string = {}
for user_id, _, step_id, __, ____, step_cost in string_passed.values:
    if user_id not in n_passed_string:
        n_passed_string[user_id] = step_cost
    else:
        n_passed_string[user_id] += step_cost

16625


In [51]:
percentage_skiped = {}
modules = sorted(list(structure.module_position.unique()))
lessons = sorted(list(structure.lesson_position.unique()))
steps_sorted = []
for module in modules:
    for lesson in lessons:
        tmp_df = structure[structure['module_position'] == module]
        needed_df = tmp_df[tmp_df['lesson_position'] == lesson]
        steps_sorted += list(map(lambda x: x[0], sorted(needed_df[['step_id', 'step_position']].values, key=lambda x: x[1])))

In [62]:
n_viewed = {}
n_viewed_test = {}
for step_id in steps_sorted:
    df = events[events['step_id'] == step_id]
    df_test = events_test[events_test['step_id'] == step_id]
    for user_id in list(df.user_id.unique()):
        if user_id not in n_viewed:
            n_viewed[user_id] = 1
        else:
            n_viewed[user_id] += 1
    for user_id in list(df_test.user_id.unique()):
        if user_id not in n_viewed_test:
            n_viewed_test[user_id] = 1
        else:
            n_viewed_test[user_id] += 1
for user_id in n_viewed:
    n_viewed[user_id] = round(n_viewed[user_id] / len(steps_sorted), 3)
for user_id in n_viewed_test:
    n_viewed_test[user_id] = round(n_viewed_test[user_id] / len(steps_sorted), 3)
print(n_viewed_test)

{8193: 0.169, 13639: 0.025, 16387: 0.085, 8196: 0.011, 5: 0.003, 9: 0.051, 8205: 0.288, 8208: 0.062, 21: 0.014, 8214: 0.008, 16409: 0.02, 8218: 0.014, 8219: 0.314, 16413: 0.023, 16414: 0.328, 8223: 0.299, 32: 0.085, 16417: 0.088, 2456: 0.006, 8227: 0.048, 16420: 0.073, 8229: 0.37, 8230: 0.734, 8233: 0.226, 4103: 0.008, 44: 0.26, 15026: 0.02, 16430: 0.023, 8239: 0.076, 16435: 0.059, 52: 0.065, 16443: 0.062, 8252: 0.051, 8254: 0.486, 8255: 0.364, 16452: 0.017, 69: 0.028, 71: 0.158, 16456: 0.068, 6839: 0.093, 16461: 0.031, 16467: 0.02, 16469: 0.006, 8280: 0.271, 8281: 0.232, 16476: 0.025, 16478: 0.048, 8287: 0.229, 98: 0.003, 8294: 0.319, 16487: 0.034, 8296: 0.023, 8297: 0.212, 8298: 0.243, 16493: 0.172, 2749: 0.006, 16497: 0.037, 16498: 0.359, 16499: 0.164, 19134: 0.003, 8310: 0.308, 119: 0.359, 10943: 0.068, 8317: 0.314, 126: 0.333, 16511: 0.003, 16516: 0.113, 16517: 0.266, 134: 0.169, 8328: 0.314, 8329: 0.054, 15331: 0.003, 140: 0.054, 16526: 0.014, 8335: 0.161, 16528: 0.136, 8338: 0.3

In [63]:
targets

,user_id,score,passed,distinction
0,1,0,0,0
1,2,82,0,0
2,3,89,0,0
3,4,9,0,0
4,6,26,0,0
5,7,119,1,1
6,8,9,0,0
7,10,3,0,0
8,11,11,0,0
9,12,0,0,0


In [ ]:
def get_cost(step_id):
    for row in structure.values:
        if row[5] == step_id:
            return row[8]

In [68]:
users_score = {}
users_score_test = {}
df = events[events['step_cost'] > 0]
df_test = events_test[events_test['step_cost'] > 0]
score_events = df[df['action'] == 'passed']
score_events_test = df_test[df_test['action'] == 'passed']
for user_id, _, __, time, ___, step_cost in score_events.values:
    if user_id not in users_score:
        users_score[user_id] = step_cost
    else:
        users_score[user_id] += step_cost
for user_id, _, __, time, ___, step_cost in score_events_test.values:
    if user_id not in users_score_test:
        users_score_test[user_id] = step_cost
    else:
        users_score_test[user_id] += step_cost

In [69]:
users_score

{3: 9,
 16388: 2,
 6: 26,
 7: 6,
 8: 9,
 16393: 52,
 10: 3,
 109: 4,
 16398: 8,
 16: 36,
 17: 15,
 16402: 10,
 546: 32,
 20: 5,
 16405: 8,
 16406: 33,
 23: 20,
 16408: 5,
 25: 75,
 27: 8,
 16412: 10,
 30: 4,
 16415: 2,
 16416: 4,
 16418: 5,
 35: 5,
 37: 6,
 38: 9,
 16424: 6,
 41: 3,
 43: 24,
 16428: 3,
 45: 30,
 46: 4,
 16431: 2,
 48: 53,
 49: 5,
 50: 2,
 16437: 5,
 16439: 22,
 16440: 12,
 57: 5,
 59: 3,
 60: 6,
 16448: 3,
 65: 2,
 16450: 5,
 70: 7,
 16457: 5,
 74: 28,
 16459: 1,
 655: 40,
 79: 22,
 16465: 1,
 84: 10,
 14: 30,
 16470: 5,
 16471: 5,
 89: 57,
 90: 14,
 91: 60,
 1094: 5,
 16477: 5,
 16482: 12,
 16483: 10,
 16484: 4,
 16485: 3,
 16486: 4,
 103: 23,
 104: 28,
 16489: 1,
 16490: 5,
 240: 3,
 18: 7,
 16494: 5,
 111: 13,
 112: 2,
 114: 1,
 19: 9,
 16501: 1,
 118: 5,
 16503: 5,
 16504: 1,
 121: 1,
 16506: 6,
 16507: 2,
 16509: 1,
 4: 9,
 127: 19,
 16515: 1,
 22: 13,
 135: 5,
 136: 4,
 137: 2,
 16523: 5,
 19138: 29,
 142: 3,
 16527: 2,
 144: 4,
 24: 19,
 16531: 2,
 149: 5,
 150:

In [73]:
ones_only = pd.read_csv('../kaggle_task/ones_only.csv')

In [75]:
ones_only.head()

,user_id,passed
0,5,1
1,9,1
2,21,1
3,26,1
4,29,1


In [76]:
users_vect = targets['user_id'].values
users_vect_test = ones_only['user_id'].values

In [ ]:
n_passed_code, n_passed_code_test = dict_passed('code')
n_passed_choice, n_passed_choice_test = dict_passed('choice')
n_passed_number, n_passed_number_test = dict_passed('number')
n_passed_string, n_passed_string_test = dict_passed('string')

1

In [104]:
df_values = []
for user_id in users_vect:
    push_vect = [user_id]
    if user_id in n_passed_code:
        push_vect.append(n_passed_code[user_id])
    else:
        push_vect.append(0)
    if user_id in n_passed_choice:
        push_vect.append(n_passed_choice[user_id])
    else:
        push_vect.append(0)
    if user_id in n_passed_number:
        push_vect.append(n_passed_number[user_id])
    else:
        push_vect.append(0)
    if user_id in n_passed_string:
        push_vect.append(n_passed_string[user_id])
    else:
        push_vect.append(0)
    if user_id in n_viewed:
        push_vect.append(n_viewed[user_id])
    else:
        push_vect.append(0)
    if user_id in users_score:
        push_vect.append(users_score[user_id])
    else:
        push_vect.append(0)
    push_vect.append(targets[targets['user_id'] == user_id].values[0][2])
    df_values.append(push_vect)

In [105]:
df_values_test = []
for user_id in users_vect_test:
    push_vect = [user_id]
    if user_id in n_passed_code_test:
        push_vect.append(n_passed_code_test[user_id])
    else:
        push_vect.append(0)
    if user_id in n_passed_choice_test:
        push_vect.append(n_passed_choice_test[user_id])
    else:
        push_vect.append(0)
    if user_id in n_passed_number_test:
        push_vect.append(n_passed_number_test[user_id])
    else:
        push_vect.append(0)
    if user_id in n_passed_string_test:
        push_vect.append(n_passed_string_test[user_id])
    else:
        push_vect.append(0)
    if user_id in n_viewed_test:
        push_vect.append(n_viewed_test[user_id])
    else:
        push_vect.append(0)
    if user_id in users_score_test:
        push_vect.append(users_score_test[user_id])
    else:
        push_vect.append(0)
    df_values_test.append(push_vect)

In [172]:
df = pd.DataFrame(data=df_values, columns=['user_id', 'n_passed_code', 'n_passed_choice', 'n_passed_number', 'n_passed_string', 'n_viewed', 'users_score', 'passed'])

In [173]:
df_test = pd.DataFrame(data=df_values_test, columns=['user_id', 'n_passed_code', 'n_passed_choice', 'n_passed_number', 'n_passed_string', 'n_viewed', 'users_score'])

In [174]:
df.drop('user_id', axis=1, inplace=True)
user_id_test = df_test['user_id']
df_test.drop('user_id', axis=1, inplace=True)

In [175]:
print(df.shape, df[df['passed'] == 1].shape)

(16625, 7) (659, 7)


In [176]:
df_tmp = pd.concat((df, df[df['passed'] == 1]))
df_ = pd.concat((df_tmp, df_tmp[df_tmp['passed'] == 1]))
df_.passed.mean()

0.1417051929900011

In [177]:
y = df_['passed']

In [178]:
df_.drop('passed', axis=1, inplace=True)

In [179]:
df_.head()

,n_passed_code,n_passed_choice,n_passed_number,n_passed_string,n_viewed,users_score
0,0,0,0,0,0.910,0
1,0,0,0,0,0.025,0
2,2,7,0,0,0.088,9
3,2,7,0,0,0.186,9
4,17,7,2,0,0.299,26


In [150]:
clf = xgb.XGBClassifier()
clf.get_params()

{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 0,
 'silent': True,
 'subsample': 1}

In [163]:
param_grid = {
    'max_depth': np.linspace(1, 15, 10, dtype=int),
    'n_estimators': np.linspace(10, 200, 15, dtype=int)
}

In [164]:
opt = GridSearchCV(clf, param_grid, scoring='f1', cv=5)

In [165]:
X = df_.values

In [166]:
opt.fit(X, y)

KeyboardInterrupt: 

In [162]:
opt.best_estimator_

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=15,
       min_child_weight=1, missing=None, n_estimators=190, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [222]:
best_clf = xgb.XGBClassifier(max_depth=15, n_estimators=800)

In [223]:
best_clf.fit(X, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=15,
       min_child_weight=1, missing=None, n_estimators=800, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [224]:
user_id_test_arr = user_id_test.values

In [225]:
y_pred = best_clf.predict(df_test.values)

In [226]:
col1 = pd.DataFrame(user_id_test_arr, columns=['user_id'])
col2 = pd.DataFrame(y_pred, columns=['passed'])
ans = pd.concat([col1, col2], axis=1)
ans.index = [int(i) for i in user_id_test_arr]
ans.index.name = 'user_id'
ans.drop('user_id', axis=1, inplace=True)
ans.to_csv('my_submission.csv')

In [191]:
def write_answer(y_pred, col_id):
    answer_df = pd.concat([col_id, y_pred], axis=1)
    answer_df.to_csv('my_submission')